In [1]:
import pandas as pd 
import numpy as np
import os

In [64]:
path_sa = r"C:\Users\LaurèneDAVID\Documents\Teaching\Educational_apps\reviews_data.csv"
data = pd.read_csv(os.path.join(path_sa,"reviews_data.csv"))

In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         850 non-null    object 
 1   location     850 non-null    object 
 2   Date         850 non-null    object 
 3   Rating       705 non-null    float64
 4   Review       850 non-null    object 
 5   Image_Links  850 non-null    object 
dtypes: float64(1), object(5)
memory usage: 40.0+ KB


In [66]:
data = data.loc[data["Review"]!="No Review Text"].reset_index(drop=True)

In [67]:
index_out = data.loc[data["Image_Links"]!="['No Images']"].index
index_in = [i for i in list(data.index) if i not in list(index_out)]
add_data = data.iloc[index_in].sample(54)

In [68]:
reviews_data_clean = pd.concat([data.iloc[index_out].reset_index(drop=True),
                                add_data.reset_index(drop=True)])

In [69]:
def clean_location(x):
    state = x.split(",")[1].strip().upper()
    if state == "CALIFORNIA":
        state = "CA"
    if state == "ALBERTA":
        state = "OTHER"

    return state

In [73]:
reviews_data_clean["location"] = reviews_data_clean["location"].apply(clean_location)
reviews_data_clean["Date"] = reviews_data_clean["Date"].apply(lambda x: x.split("Reviewed")[1].strip())

In [74]:
# FINAL DATASET
reviews_data_final = reviews_data_clean.loc[reviews_data_clean["location"].isin(["CA","TX","PA","OR","MO","MN"])]

In [75]:
from datetime import datetime

def format_date(date):
    if "Jan." in date:
        date = date.replace("Jan.","January")
    if "Feb." in date:
        date = date.replace("Feb.", "February")
    if "Aug." in date:
        date = date.replace("Aug.", "August")
    if "Sept." in date:
        date = date.replace("Sept.", "September")
    if "Oct." in date:
        date = date.replace("Oct.", "October")
    if "Nov." in date: 
        date = date.replace("Nov.", "November")
    if "Dec." in date:
        date = date.replace("Dec.", "December")

    date = date.replace(",","")

    parsed_date = datetime.strptime(date, "%B %d %Y")

    # Format the date as MM/DD/YYYY
    formatted_date = parsed_date.strftime("%m-%d-%Y")
    
    return formatted_date

In [76]:
reviews_data_final["Date"] = pd.to_datetime(reviews_data_final["Date"].apply(format_date))

C:\Users\LaurèneDAVID\AppData\Local\Temp\ipykernel_3128\1306421503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_data_final["Date"] = pd.to_datetime(reviews_data_final["Date"].apply(format_date))


In [77]:
dict_states = {"CA":"California",
               "TX":"Texas",
               "PA":"Pennsylvania",
               "OR":"Oregon",
               "MO":"Missouri",
               "MN":"Minnesota"}

In [78]:
reviews_data_final["location"] = reviews_data_final["location"].map(dict_states)

C:\Users\LaurèneDAVID\AppData\Local\Temp\ipykernel_3128\3901736406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_data_final["location"] = reviews_data_final["location"].map(dict_states)


In [79]:
reviews_data_final.drop(columns=["name"],inplace=True)

C:\Users\LaurèneDAVID\AppData\Local\Temp\ipykernel_3128\332411794.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_data_final.drop(columns=["name"],inplace=True)


In [80]:
import re 

def clean_text(text):
    pattern_punct = r"[^\w\s.',:/]"
    pattern_date = r'\b\d{1,2}/\d{1,2}/\d{2,4}\b'

    text = text.lower()
    text = re.sub(pattern_date, '', text)
    text = re.sub(pattern_punct, '', text)
    text = text.replace("ggg","g")
    text = text.replace("  "," ")

    return text

In [81]:
clean_image_urls = [url.replace("['","").replace("']","").replace("',"," ").replace("'",'') for url in reviews_data_final["Image_Links"].to_list()]
clean_image_urls = [elem.split("  ") for elem in clean_image_urls]

In [82]:
images_1 = []
images_2 = []

for elem in clean_image_urls:

    if elem[0] != "No Images":
        images_1.append(elem[0])
    else: 
        images_1.append(np.nan)

    if len(elem) == 2:
        images_2.append(elem[1])
    else:
        images_2.append(np.nan)

In [83]:
reviews_data_final["Image 1"] = images_1
reviews_data_final["Image 2"] = images_2

C:\Users\LaurèneDAVID\AppData\Local\Temp\ipykernel_3128\464558646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_data_final["Image 1"] = images_1
C:\Users\LaurèneDAVID\AppData\Local\Temp\ipykernel_3128\464558646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_data_final["Image 2"] = images_2


In [84]:
reviews_data_final.rename({"location":"State"}, axis=1, inplace=True)
reviews_data_final.drop(columns=["Image_Links"])
reviews_data_final = reviews_data_final[["Date","State","Review","Rating","Image 1","Image 2"]]

C:\Users\LaurèneDAVID\AppData\Local\Temp\ipykernel_3128\1924389021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_data_final.rename({"location":"State"}, axis=1, inplace=True)


In [86]:
reviews_data_final["Year"] = reviews_data_final["Date"].dt.year
reviews_data_final.insert(0, "ID", [f"{i}" for i in np.arange(1, len(reviews_data_final)+1)])

In [89]:
reviews_data_final.to_pickle(os.path.join(path_sa,"reviews_raw.pkl"))

In [90]:
reviews_data_final_clean = reviews_data_final.copy()
reviews_data_final_clean["Review"] = reviews_data_final_clean["Review"].apply(clean_text)
# reviews_data_final.to_pickle(os.path.join(path_sa,"reviews_clean.pkl"))

In [91]:
from pysentimiento import create_analyzer

list_reviews = reviews_data_final_clean["Review"].to_list()
sentiment_analyzer = create_analyzer(task="sentiment", lang="en")
predictions = []
positive = []
negative = []
neutral = []

for review in list_reviews:
    #if review.split(" ")
    q = sentiment_analyzer.predict(review)

    predictions.append(q.output)
    positive.append(q.probas["POS"])
    negative.append(q.probas["NEG"])
    neutral.append(q.probas["NEU"])

# Results
df_results = reviews_data_final_clean.copy()
df_results["Result"] = predictions
df_results["Result"] = df_results["Result"].map({"NEU":"Neutral", "NEG":"Negative", "POS":"Positive"})
df_results["Negative"] = np.round(np.array(negative)*100)
df_results["Neutral"] = np.round(np.array(neutral)*100)
df_results["Positive"] = np.round(np.array(positive)*100)

In [93]:
df_results.to_pickle(os.path.join(path_sa,"reviews_results.pkl"))